In [1]:
import sys
sys.path
backend_path = '../../backend'
if backend_path not in sys.path:
    sys.path.append(backend_path)

In [2]:
sys.path

['/home/peter-legion-wsl2/peter-projects/contract-queue/nbs',
 '/usr/lib/python310.zip',
 '/usr/lib/python3.10',
 '/usr/lib/python3.10/lib-dynload',
 '',
 '/home/peter-legion-wsl2/peter-projects/contract-queue/.venv/lib/python3.10/site-packages',
 '../../backend']